## Mongo as data prep

This is some discovery work to start to identify how we can execute data preprocessing jobs in MongoDB.

In [3]:
%pprint

Pretty printing has been turned ON


## Necessary dependecies

In [1]:
%pip install docker pymongo pandas

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


## Run a mongo container

In [55]:
import os
import docker

docker_client = docker.from_env()

mongo_container = docker_client.containers.run(
    'mongo:latest',
    detach=True,
    name='mongo-data-prep',
    remove=True,
    ports={'27017/tcp': 27018},
    mem_limit='512M',
    volumes={
        os.path.join(os.getcwd(), 'db'): {'bind': '/data/db', 'mode': 'rw'},
        os.path.join(os.getcwd(), 'mongo.conf'): {'bind': '/etc/mongo.conf', 'mode': 'ro'}
    }
)

To get its state:

In [55]:
mongo_container.logs()

b'2020-05-02T12:46:04.050+0000 I  CONTROL  [main] Automatically disabling TLS 1.0, to force-enable TLS 1.0 specify --sslDisabledProtocols \'none\'\n2020-05-02T12:46:04.057+0000 W  ASIO     [main] No TransportLayer configured during NetworkInterface startup\n2020-05-02T12:46:04.057+0000 I  CONTROL  [initandlisten] MongoDB starting : pid=1 port=27017 dbpath=/data/db 64-bit host=ca620174dbc5\n2020-05-02T12:46:04.057+0000 I  CONTROL  [initandlisten] db version v4.2.5\n2020-05-02T12:46:04.057+0000 I  CONTROL  [initandlisten] git version: 2261279b51ea13df08ae708ff278f0679c59dc32\n2020-05-02T12:46:04.057+0000 I  CONTROL  [initandlisten] OpenSSL version: OpenSSL 1.1.1  11 Sep 2018\n2020-05-02T12:46:04.057+0000 I  CONTROL  [initandlisten] allocator: tcmalloc\n2020-05-02T12:46:04.057+0000 I  CONTROL  [initandlisten] modules: none\n2020-05-02T12:46:04.057+0000 I  CONTROL  [initandlisten] build environment:\n2020-05-02T12:46:04.057+0000 I  CONTROL  [initandlisten]     distmod: ubuntu1804\n2020-05-

To stop it:

In [52]:
mongo_container.stop()

## Connect using PyMongo

In [56]:
from pymongo import MongoClient
mongo_client = MongoClient('127.0.0.1', 27018)
db = mongo_client['data-prep']

In [4]:
mongo_client.server_info()

{'version': '4.2.5',
 'gitVersion': '2261279b51ea13df08ae708ff278f0679c59dc32',
 'modules': [],
 'allocator': 'tcmalloc',
 'javascriptEngine': 'mozjs',
 'sysInfo': 'deprecated',
 'versionArray': [4, 2, 5, 0],
 'openssl': {'running': 'OpenSSL 1.1.1  11 Sep 2018',
  'compiled': 'OpenSSL 1.1.1  11 Sep 2018'},
 'buildEnvironment': {'distmod': 'ubuntu1804',
  'distarch': 'x86_64',
  'cc': '/opt/mongodbtoolchain/v3/bin/gcc: gcc (GCC) 8.2.0',
  'ccflags': '-fno-omit-frame-pointer -fno-strict-aliasing -ggdb -pthread -Wall -Wsign-compare -Wno-unknown-pragmas -Winvalid-pch -Werror -O2 -Wno-unused-local-typedefs -Wno-unused-function -Wno-deprecated-declarations -Wno-unused-const-variable -Wno-unused-but-set-variable -Wno-missing-braces -fstack-protector-strong -fno-builtin-memcmp',
  'cxx': '/opt/mongodbtoolchain/v3/bin/g++: g++ (GCC) 8.2.0',
  'cxxflags': '-Woverloaded-virtual -Wno-maybe-uninitialized -fsized-deallocation -std=c++17',
  'linkflags': '-pthread -Wl,-z,now -rdynamic -Wl,--fatal-war

A note on why we need why the docker memory limit is not enough.
We can see below that the databse thinks it can use all the host memory (see note in https://docs.mongodb.com/v3.2/reference/configuration-options/#storage.wiredTiger.engineConfig.cacheSizeGB).
We need to limit manually how much memory it can take using the mongo.conf file.

In [3]:
db.command('hostInfo')['system']

{'currentTime': datetime.datetime(2020, 5, 3, 8, 34, 45, 646000),
 'hostname': '3b67778f62f8',
 'cpuAddrSize': 64,
 'memSizeMB': 15829,
 'memLimitMB': 512,
 'numCores': 8,
 'cpuArch': 'x86_64',
 'numaEnabled': False}

## Creating some random data

In [34]:
from datetime import date
import pandas as pd
from numpy.random import default_rng

rng = default_rng()

date_range = pd.date_range(
    end=date.today(),
    start=date.fromisoformat('1970-01-01'),
    freq='min',
    name='moment',
)
print(f'There is a total of {date_range.size} rows')
 
# Generate only a chunk at a time to avoid having thbe whole dataframe in memory
def generate_sample_chunk_df(chunk_size=100000):
    rows_count = date_range.size
    chunks_count = int(rows_count / chunk_size)
    print(f'There is {chunks_count} chunks')

    for i in range(chunks_count + 1):
        slice_begin = i * chunk_size
        if i <= chunks_count:
            slice_end = slice_begin + chunk_size - 1
        else:  # For the last chunk
            slice_end = slice_begin + 1 + rows_count % chunk_size

        chunk_date_range = date_range[slice_begin : slice_end]

        df = pd.DataFrame(index=chunk_date_range, columns=[])

        df['quantity'] = rng.integers(0, 1000, chunk_date_range.size)
        df['color'] = rng.choice(['black', 'white', 'purple', 'cyan'], chunk_date_range.size)

        # Creates the moment column from the index
        df.reset_index(inplace=True)

        # Creates an index column
        df.reset_index(inplace=True)
        df['index'] += slice_begin

        yield df

# The first chunk
g = generate_sample_chunk_df()
next(g)

# Wanna see the last one? use reversed(range(...)) in the generator

There is a total of 26474401 rows
There is 26474 chunks


### Inserting in mongo

In [8]:
%%time
db.input_data.drop()

i = 0
for chunk in generate_sample_chunk_df():
    i += 1
    print(f'Inserting chunk {i}...')
    db.input_data.insert_many(chunk.to_dict(orient='record'))

There is 264 chunks
Inserting chunk 1...
Inserting chunk 2...
Inserting chunk 3...
Inserting chunk 4...
Inserting chunk 5...
Inserting chunk 6...
Inserting chunk 7...
Inserting chunk 8...
Inserting chunk 9...
Inserting chunk 10...
Inserting chunk 11...
Inserting chunk 12...
Inserting chunk 13...
Inserting chunk 14...
Inserting chunk 15...
Inserting chunk 16...
Inserting chunk 17...
Inserting chunk 18...
Inserting chunk 19...
Inserting chunk 20...
Inserting chunk 21...
Inserting chunk 22...
Inserting chunk 23...
Inserting chunk 24...
Inserting chunk 25...
Inserting chunk 26...
Inserting chunk 27...
Inserting chunk 28...
Inserting chunk 29...
Inserting chunk 30...
Inserting chunk 31...
Inserting chunk 32...
Inserting chunk 33...
Inserting chunk 34...
Inserting chunk 35...
Inserting chunk 36...
Inserting chunk 37...
Inserting chunk 38...
Inserting chunk 39...
Inserting chunk 40...
Inserting chunk 41...
Inserting chunk 42...
Inserting chunk 43...
Inserting chunk 44...
Inserting chunk 45...

During this long operation, we can monitor that the mongo container never reach its limit:
`docker stats`.

In [6]:
db.input_data.count_documents({})

26472697

In [4]:
db.input_data.data_size()

TypeError: 'Collection' object is not callable. If you meant to call the 'data_size' method on a 'Collection' object it is failing because no such method exists.

### Running an aggregation

A simple aggregation on a subset (100 documents) is fast:

In [17]:
match_all = {
    '$match': {}
}

limit_100 = {
    '$limit': 100
}

add_year_field = {
    '$addFields': {
        'year': {'$year': '$moment'},
    }
}

aggregate_by_year = {
    '$group': {
        '_id': {
            'year': '$year'
        },
        'quantity': {'$avg': '$quantity'},
        'color': {'$first': '$color'},
        'start_index': {'$first': '$index'},
        'end_index': {'$last': '$index'},
    }
}

In [23]:
%%time

list(db.input_data.aggregate([
    match_all,
    limit_100,
    add_year_field,
    aggregate_by_year,
]))

CPU times: user 6.07 ms, sys: 0 ns, total: 6.07 ms
Wall time: 6.36 ms


[{'_id': {'year': 1970},
  'quantity': 507.17,
  'color': 'cyan',
  'start_index': 0,
  'end_index': 99}]

But without the limit, it takes a while but does works without crashing anything:

In [24]:
%%time

results = db.input_data.aggregate([
    match_all,
    add_year_field,
    aggregate_by_year,
])

CPU times: user 9.58 ms, sys: 6.6 ms, total: 16.2 ms
Wall time: 51.6 s


In [28]:
pd.DataFrame(list(results))

,_id,quantity,color,start_index,end_index
0,{'year': 1985},498.973548,cyan,7889760,8415359
1,{'year': 1977},498.434233,white,3682080,4207679
2,{'year': 2001},499.745214,cyan,16305120,16830719
3,{'year': 2009},499.932941,white,20512800,21038399
4,{'year': 1998},500.043629,black,14726880,15252479
5,{'year': 2015},500.070008,cyan,23667840,24193439
6,{'year': 2020},498.643856,purple,26297280,26472960
7,{'year': 1988},498.907750,cyan,9466560,9993599
8,{'year': 2013},499.772079,cyan,22616640,23142239
9,{'year': 2016},498.678129,cyan,24193440,24720479


Note the weird order of results! MongoDB docs states that documents are not sorted after a `$group` stage.

Conclusion: the memory limit really protects the mongo process. It takes all the CPU time of the container, so it'll be required to test if only one core is used in full or if the whole mongo server will become unresponsive.

Let's try an aggregation with many more documents:

In [38]:
add_hourly_field = {
    '$addFields': {
        'year': {'$year': '$moment'},
        'month': {'$month': '$moment'},
        'day': {'$dayOfMonth': '$moment'},
        'hour': {'$hour': '$moment'},
    }
}

aggregate_by_hour = {
    '$group': {
        '_id': {
            'year': '$year',
            'month': '$month',
            'day': '$day',
            'hour': '$hour',
        },
        'quantity': {'$avg': '$quantity'},
        'color': {'$first': '$color'},
        'start_index': {'$first': '$index'},
        'end_index': {'$last': '$index'},
    }
}

In [53]:
%%time
results = db.input_data.aggregate([
    match_all,
    add_hourly_field,
    aggregate_by_hour,
])

ServerSelectionTimeoutError: 127.0.0.1:27018: [Errno 111] Connection refused

Fails with `OperationFailure: Exceeded memory limit for $group, but didn't allow external sort. Pass allowDiskUse:true to opt in.`

Two things next:
- first, let's enable the `allowDiskUse`
- second, let's write the output in another collection

We do not want data to get out of the db to our pymongo client, for it will deplete our host's memory.

In [39]:
write_in_output_collection = {
    '$out': 'output_data'
}

In [57]:
%%time

results = db.input_data.aggregate([
    match_all,
    add_hourly_field,
    aggregate_by_hour,
    write_in_output_collection
], allowDiskUse=True)

CPU times: user 34.7 ms, sys: 10.9 ms, total: 45.6 ms
Wall time: 1min 33s


In [47]:
r = db.output_data.find({})
r[0]

{'_id': {'year': 1970, 'month': 1, 'day': 1, 'hour': 0},
 'quantity': 501.65,
 'color': 'cyan',
 'start_index': 0,
 'end_index': 59}

Note: this seems to indicate we could monitor the process by having a second command querying for the output results: https://stackoverflow.com/questions/22725814/view-progress-of-long-running-mongodb-aggregation-job